In [ ]:
# =============================================================================
# 🚀 DATABASE RECOMMENDATION API
# =============================================================================

# 📦 DEPENDENCIES INSTALLATION
print("🔄 Installing dependencies...")

# Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh
# Install required packages
!pip install fastapi uvicorn pyngrok requests
!pip install "pydantic[email]" jinja2 python-multipart
# Import libraries
import subprocess
import threading
import time
import requests
import json
import logging
import re
import uuid
from typing import Dict, List, Optional, Any, Union
from pydantic import BaseModel, Field
from fastapi import FastAPI, Body, HTTPException
import uvicorn
from pyngrok import ngrok
from enum import Enum

# =============================================================================
# 🛠️ LOGGING CONFIGURATION
# =============================================================================

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('database_recommendation.log')
    ]
)

logger = logging.getLogger("database_recommendation")

# =============================================================================
# 📊 DATA MODELS
# =============================================================================

class DatabaseType(str, Enum):
    POSTGRES = "postgres"
    CLICKHOUSE = "clickhouse"
    MYSQL = "mysql"
    MONGODB = "mongodb"
    REDIS = "redis"
    ELASTICSEARCH = "elasticsearch"
    CASSANDRA = "cassandra"

class LanguageType(str, Enum):
    RU = "ru"
    EN = "en"

class DatabaseRecommendationRequest(BaseModel):
    use_case: str = Field(..., description="Описание варианта использования")
    data_characteristics: Dict[str, Any] = Field(..., description="Характеристики данных (может содержать schema_input)")
    performance_requirements: Dict[str, Any] = Field(None, description="Требования к производительности")
    input_language: LanguageType = Field(default=LanguageType.RU, description="Язык входных данных")
    output_language: LanguageType = Field(default=LanguageType.RU, description="Язык ответа")

class DatabaseRecommendation(BaseModel):
    database: str = Field(..., description="Рекомендуемая база данных")
    reason: str = Field(..., description="Обоснование рекомендации")
    pros: List[str] = Field(..., description="Преимущества")
    cons: List[str] = Field(..., description="Недостатки")
    suitability_score: int = Field(..., description="Оценка применимости от 1 до 100")

class DatabaseRecommendationResponse(BaseModel):
    recommended_databases: List[DatabaseRecommendation] = Field(..., description="Список рекомендованных БД")
    storage_strategy: str = Field(..., description="Стратегия хранения данных")
    data_modeling_advice: List[str] = Field(..., description="Советы по моделированию данных")
    performance_optimizations: List[str] = Field(..., description="Оптимизации производительности")
    success: bool = Field(..., description="Статус выполнения")

# =============================================================================
# 🎯 OLLAMA SETUP
# =============================================================================

def setup_ollama():
    """Start Ollama server in background"""
    logger.info("🔧 Starting Ollama server...")

    def run_ollama():
        try:
            subprocess.run(["ollama", "serve"], check=True)
        except subprocess.CalledProcessError as e:
            logger.error(f"Ollama startup error: {e}")

    ollama_thread = threading.Thread(target=run_ollama, daemon=True)
    ollama_thread.start()
    time.sleep(10)

    for i in range(3):
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=30)
            if response.status_code == 200:
                logger.info("✅ Ollama server started successfully")
                return True
        except:
            logger.warning(f"⚠️ Ollama connection attempt {i+1}/3 failed, retrying...")
            time.sleep(5)

    logger.error("❌ Could not connect to Ollama server")
    return False

def download_model():
    """Download Mistral 7B model"""
    model_name = "mistral:7b"

    logger.info(f"📥 Downloading {model_name}...")
    try:
        process = subprocess.run(
            ["ollama", "pull", model_name],
            timeout=1800,
            capture_output=True,
            text=True
        )
        if process.returncode == 0:
            logger.info(f"✅ {model_name} downloaded successfully")
            return True
        else:
            logger.warning(f"⚠️ {model_name} download issues: {process.stderr}")
            return False
    except subprocess.TimeoutExpired:
        logger.warning(f"⚠️ {model_name} download timeout")
        return False
    except Exception as e:
        logger.error(f"❌ Error downloading {model_name}: {e}")
        return False

# Initialize Ollama
setup_ollama()
logger.info("📥 Loading Mistral 7B model...")
download_model()
time.sleep(10)

# =============================================================================
# 🤖 LLM CLIENT
# =============================================================================

class LLMClient:
    def __init__(self):
        self.base_url = "http://localhost:11434"
        self.timeout = 300
        self.model = "mistral:7b"

    def generate_database_recommendation(self, prompt: str) -> str:
        """Generate database recommendation using Mistral model"""
        return self._generate_content(prompt)

    def _generate_content(self, prompt: str) -> str:
        """Generic content generation method"""
        try:
            payload = {
                "model": self.model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.1,
                    "top_k": 1,
                    "top_p": 0.1
                }
            }

            logger.info(f"🤖 Generating database recommendation with {self.model}...")
            response = requests.post(
                f"{self.base_url}/api/generate",
                json=payload,
                timeout=self.timeout
            )

            if response.status_code == 200:
                result = response.json()
                content = result.get("response", "").strip()

                # Clean up the response
                content = re.sub(r'```json\s*', '', content)
                content = re.sub(r'```\s*', '', content)
                content = content.strip()

                logger.info(f"✅ Recommendation generated successfully ({len(content)} chars)")
                return content
            else:
                logger.error(f"❌ LLM API error: {response.status_code} - {response.text}")
                return f"ERROR: LLM API returned {response.status_code}"

        except requests.exceptions.Timeout:
            logger.error("❌ LLM request timeout")
            return "ERROR: Request timeout"
        except Exception as e:
            logger.error(f"❌ LLM request failed: {str(e)}")
            return f"ERROR: {str(e)}"

# =============================================================================
# 🎯 DATABASE RECOMMENDATION SERVICE
# =============================================================================

class DatabaseRecommendationService:
    def __init__(self):
        self.llm_client = LLMClient()

    def recommend_databases(self, request: DatabaseRecommendationRequest) -> DatabaseRecommendationResponse:
        """Generate database recommendations based on use case and data characteristics"""
        logger.info("🎯 Starting database recommendation analysis...")

        try:
            # Create comprehensive prompt for database recommendation
            prompt = self._create_recommendation_prompt(request)

            # Generate recommendation using LLM
            recommendation_text = self.llm_client.generate_database_recommendation(prompt)

            # Parse the response into structured format
            parsed_recommendation = self._parse_recommendation_response(recommendation_text, request.output_language)

            logger.info("✅ Database recommendation completed successfully")
            return parsed_recommendation

        except Exception as e:
            logger.error(f"❌ Database recommendation failed: {str(e)}")
            # Return fallback recommendation
            return self._get_fallback_recommendation(request.output_language)

    def _create_recommendation_prompt(self, request: DatabaseRecommendationRequest) -> str:
        """Create detailed prompt for database recommendation"""

        # Extract schema information if present
        schema_info = ""
        if "inputs" in request.data_characteristics:
            inputs = request.data_characteristics.get("inputs", [])
            if inputs:
                schema_data = inputs[0].get("schema", {})
                if isinstance(schema_data, dict) and "fields" in schema_data:
                    schema_info = f"Schema fields: {len(schema_data['fields'])}"
                elif isinstance(schema_data, str) and "CREATE TABLE" in schema_data.upper():
                    schema_info = "DDL schema provided with CREATE TABLE statements"

        language_instruction = self._get_language_instruction(request.input_language, request.output_language)

        prompt = f"""
{language_instruction}

DATABASE RECOMMENDATION ANALYSIS REQUEST:

USE CASE:
{request.use_case}

DATA CHARACTERISTICS:
{json.dumps(request.data_characteristics, indent=2, ensure_ascii=False)}
{schema_info}

PERFORMANCE REQUIREMENTS:
{json.dumps(request.performance_requirements, indent=2, ensure_ascii=False)}

ANALYSIS CRITERIA:
1. Data Volume & Velocity - размер данных и скорость поступления
2. Query Patterns - типы запросов (OLTP, OLAP, аналитические)
3. Data Structure - структурированные, полуструктурированные, неструктурированные
4. Consistency Requirements - требования к консистентности
5. Scalability Needs - потребности в масштабировании
6. Operational Complexity - операционная сложность

AVAILABLE DATABASES TO CONSIDER:
- PostgreSQL - универсальная реляционная БД, ACID, JSON поддержка
- ClickHouse - колоночная БД для аналитики, высокая производительность
- MySQL - популярная реляционная БД, веб-приложения
- MongoDB - документная БД, гибкая схема
- Redis - in-memory БД, кэширование, очереди
- Elasticsearch - поиск и аналитика, полнотекстовый поиск
- Cassandra - распределенная БД, высокая доступность

REQUIRED RESPONSE FORMAT (JSON):
{{
  "recommended_databases": [
    {{
      "database": "database_name",
      "reason": "detailed explanation",
      "pros": ["advantage1", "advantage2"],
      "cons": ["disadvantage1", "disadvantage2"],
      "suitability_score": 85
    }}
  ],
  "storage_strategy": "overall storage approach",
  "data_modeling_advice": ["advice1", "advice2"],
  "performance_optimizations": ["optimization1", "optimization2"],
  "success": true
}}

IMPORTANT:
- Provide 2-3 most suitable databases with scores 70-100
- Be specific and practical in recommendations
- Avoid duplicate information across different sections
- Focus on real-world applicability
- Consider both technical and operational aspects
- Provide actionable advice

RETURN ONLY VALID JSON WITHOUT ANY ADDITIONAL TEXT.
"""

        return prompt

    def _get_language_instruction(self, input_lang: LanguageType, output_lang: LanguageType) -> str:
        """Get language instruction for the prompt"""
        if input_lang == LanguageType.EN and output_lang == LanguageType.EN:
            return "Please provide the response in English. Keep technical terms in English."
        elif input_lang == LanguageType.RU and output_lang == LanguageType.EN:
            return "The input is in Russian but please provide the response in English. Keep technical terms in English."
        elif input_lang == LanguageType.EN and output_lang == LanguageType.RU:
            return "The input is in English but please provide the response in Russian. Keep technical terms in English."
        else:
            return "Please provide the response in Russian. Keep technical terms in English."

    def _parse_recommendation_response(self, response_text: str, output_language: LanguageType) -> DatabaseRecommendationResponse:
        """Parse LLM response into structured format"""
        try:
            # Try to extract JSON from response
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                data = json.loads(json_str)

                # Validate and convert to response model
                return DatabaseRecommendationResponse(
                    recommended_databases=[
                        DatabaseRecommendation(
                            database=db["database"],
                            reason=db["reason"],
                            pros=db["pros"],
                            cons=db["cons"],
                            suitability_score=db["suitability_score"]
                        ) for db in data["recommended_databases"]
                    ],
                    storage_strategy=data["storage_strategy"],
                    data_modeling_advice=data["data_modeling_advice"],
                    performance_optimizations=data["performance_optimizations"],
                    success=data["success"]
                )
            else:
                raise ValueError("No JSON found in response")

        except Exception as e:
            logger.warning(f"Failed to parse LLM response, using fallback: {e}")
            return self._get_fallback_recommendation(output_language)

    def _get_fallback_recommendation(self, output_language: LanguageType) -> DatabaseRecommendationResponse:
        """Provide fallback recommendation when LLM fails"""
        if output_language == LanguageType.EN:
            return DatabaseRecommendationResponse(
                recommended_databases=[
                    DatabaseRecommendation(
                        database="PostgreSQL",
                        reason="Universal relational database with excellent ACID compliance and JSON support",
                        pros=["Strong consistency", "Rich feature set", "Great community support", "JSON and spatial data support"],
                        cons=["Can require more tuning for high-scale analytics", "Less optimized for columnar operations than ClickHouse"],
                        suitability_score=80
                    ),
                    DatabaseRecommendation(
                        database="ClickHouse",
                        reason="Column-oriented database optimized for analytical queries and large-scale data processing",
                        pros=["Excellent for analytical workloads", "High compression rates", "Fast aggregations"],
                        cons=["Weaker for transactional workloads", "Less mature ecosystem than PostgreSQL"],
                        suitability_score=75
                    )
                ],
                storage_strategy="Consider using PostgreSQL for transactional data and ClickHouse for analytical workloads in a dual-database architecture",
                data_modeling_advice=[
                    "Normalize data for transactional workloads",
                    "Use denormalized structures for analytical queries",
                    "Consider partitioning for large datasets"
                ],
                performance_optimizations=[
                    "Use appropriate indexes for query patterns",
                    "Consider connection pooling for high concurrency",
                    "Monitor and tune database configuration parameters"
                ],
                success=True
            )
        else:
            return DatabaseRecommendationResponse(
                recommended_databases=[
                    DatabaseRecommendation(
                        database="PostgreSQL",
                        reason="Универсальная реляционная СУБД с отличной поддержкой ACID и JSON",
                        pros=["Сильная консистентность", "Богатый функционал", "Отличная поддержка сообщества", "Поддержка JSON и пространственных данных"],
                        cons=["Требует настройки для аналитики больших объемов", "Менее оптимизирована для колоночных операций чем ClickHouse"],
                        suitability_score=80
                    ),
                    DatabaseRecommendation(
                        database="ClickHouse",
                        reason="Колоночная СУБД оптимизирована для аналитических запросов и обработки больших данных",
                        pros=["Отлично подходит для аналитических нагрузок", "Высокий уровень сжатия", "Быстрые агрегации"],
                        cons=["Слабее для транзакционных нагрузок", "Менее зрелая экосистема чем PostgreSQL"],
                        suitability_score=75
                    )
                ],
                storage_strategy="Рекомендуется использовать PostgreSQL для транзакционных данных и ClickHouse для аналитических нагрузок в архитектуре с двумя БД",
                data_modeling_advice=[
                    "Нормализуйте данные для транзакционных нагрузок",
                    "Используйте денормализованные структуры для аналитических запросов",
                    "Рассмотрите партиционирование для больших наборов данных"
                ],
                performance_optimizations=[
                    "Используйте соответствующие индексы для шаблонов запросов",
                    "Рассмотрите пул соединений для высокой конкурентности",
                    "Мониторьте и настраивайте параметры конфигурации БД"
                ],
                success=True
            )

# =============================================================================
# 🚀 FASTAPI APPLICATION
# =============================================================================

app = FastAPI(
    title="Database Recommendation API",
    description="API для рекомендации подходящих баз данных на основе варианта использования",
    version="1.0"
)

database_recommendation_service = DatabaseRecommendationService()

@app.post("/recommend-database", response_model=DatabaseRecommendationResponse)
async def recommend_database(request: DatabaseRecommendationRequest = Body(...)):
    """Рекомендация подходящих баз данных на основе варианта использования и характеристик данных"""
    logger.info("🎯 Starting database recommendation analysis")

    try:
        recommendation = database_recommendation_service.recommend_databases(request)

        logger.info("✅ Database recommendation completed successfully")
        return recommendation

    except Exception as e:
        logger.error(f"❌ Database recommendation failed: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Database recommendation failed: {str(e)}")

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {
        "status": "healthy",
        "service": "Database Recommendation API",
        "version": "1.0",
        "supported_databases": [db.value for db in DatabaseType],
        "model": "mistral:7b"
    }

# =============================================================================
# 🚀 SERVER STARTUP
# =============================================================================

def run_server():
    """Run FastAPI server"""
    uvicorn.run(app, host="0.0.0.0", port=9008)

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

logger.info("⏳ Starting Database Recommendation server...")
time.sleep(5)


# =============================================================================
# 🔗 NGROK TUNNEL
# =============================================================================

NGROK_AUTH_TOKEN = "33VYITl8jw9h78PdWIl1hgXJDk0_umeC43CgfdBrAnnEjBQu"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

try:
    public_tunnel = ngrok.connect(9008, bind_tls=True)
    public_url = public_tunnel.public_url
except Exception as e:
    logger.error(f"❌ Ngrok error: {e}")
    print(f"❌ Ngrok failed: {e}")
    print("📡 Server running locally on http://localhost:9008")

In [ ]:
# 🔄 KEEP-ALIVE LOOP FOR SERVER AND SESSION
import time
import threading

def keep_alive():
    """Простой цикл для поддержания активности сервера и сессии"""
    while True:
        print(f"🕒 Session alive at {time.strftime('%H:%M:%S')}")
        time.sleep(60)  # Сообщение каждую минуту

# Запускаем в фоновом потоке
keep_alive_thread = threading.Thread(target=keep_alive, daemon=True)
keep_alive_thread.start()

print("✅ Keep-alive loop started - server will stay active")